In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import confusion_matrix,classification_report
from xgboost import XGBClassifier
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
import time

ALL = False
NO_SMOTE = True
TEST_RATIO = 0.3

dtypes = {
    'msisdn': 'str',
    'start_time': 'str',
    'end_time': 'str',
    'call_event': 'category',
    'other_party': 'str',
    'ismultimedia': 'category',
    'home_area_code': 'str',
    'visit_area_code': 'str',
    'called_home_code': 'str',
    'called_code': 'str',
    'a_serv_type': 'int',
    'long_type1': 'int',
    'roam_type': 'int',
    'a_product_id': 'str',
    'open_datetime': 'str',
    'call_duration': 'int32',
    'cfee': 'float64',
    'lfee': 'float64',
    'hour': 'int8',
    'dayofweek': 'int',
    'phone1_type': 'int',
    'phone2_type': 'int',
    'phone1_loc_city': 'str',
    'phone1_loc_province': 'str',
    'phone2_loc_city': 'str',
    'phone2_loc_province': 'str',
    'update_time': 'str',
    'date': 'str',
    'date_c': 'str'
}

# 判断 processed 文件夹是否存在
import os
if not os.path.exists('../self_data/processed'):
    print("Creating processed data folder...")
    # 读取CSV文件
    labeled_data = pd.read_csv('../self_data/sorted_trainSet_res_with_head.csv', dtype=dtypes)
    labels = pd.read_csv('../self_data/trainSet_ans_with_head.csv', dtype=dtypes)

    validation_data = pd.read_csv('../self_data/sorted_validationSet_res_with_head.csv', dtype=dtypes)

    # 按照 msisdn 切分 train_data 和 test_data
    train_data_msisdn, test_data_msisdn = train_test_split(labels['msisdn'], test_size=TEST_RATIO, random_state=42, stratify=labels['is_sa'])
    train_data = labeled_data[labeled_data['msisdn'].isin(train_data_msisdn)]
    train_labels = labels[labels['msisdn'].isin(train_data_msisdn)]
    assert len(train_data['msisdn'].unique()) == len(train_data_msisdn)

    test_data = labeled_data[labeled_data['msisdn'].isin(test_data_msisdn)]
    test_labels = labels[labels['msisdn'].isin(test_data_msisdn)]
    assert len(test_data['msisdn'].unique()) == len(test_data_msisdn)


    # # 遍历 groupby('msisdn') 的结果，对每个 msisdn 进行数据增强
    # # ------
    from tqdm import tqdm
    import os
    import sys
    from utils.augmentation import Augmentation

    addition_train_data = []
    addition_train_labels = []

    times = 1
    # ratio_range = 0.1
    ratio_range = (0.01, 0.2)
    pbar = tqdm(train_data.groupby('msisdn'))
    for msisdn, group in pbar:
        if msisdn == 0:
            continue
        # print(f"Augmenting msisdn {msisdn}")
        pbar.set_description(f"Augmenting msisdn {msisdn}")
        label = train_labels[train_labels['msisdn'] == msisdn].iloc[0]['is_sa']
        aug = Augmentation(group, label, 'msisdn', 'is_sa')
        # 对正负样本进行平衡 样本比 1:4
        if label == 1:
            res_df, res_labels = aug.times(ratio=ratio_range, times=9*times, method='mask')

            addition_train_data.append(res_df)
            addition_train_labels.append(res_labels)

            # res_df, res_labels = aug.times(window_size=100, step_size=80, times=1, method='sliding_window')

            # addition_train_data.append(res_df)
            # addition_train_labels.append(res_labels)
        # else:
        #     res_df, res_labels = aug.times(ratio=ratio_range, times=times, method='mask')

        #     addition_train_data.append(res_df)
        #     addition_train_labels.append(res_labels)

            # res_df, res_labels = aug.times(window_size=100, step_size=80, times=1, method='sliding_window')

            # addition_train_data.append(res_df)
            # addition_train_labels.append(res_labels)
            
    addition_train_data = pd.concat(addition_train_data)
    addition_train_labels = pd.concat(addition_train_labels)

    # 将新数据加入到train_data中
    train_data = pd.concat([train_data, addition_train_data], ignore_index=True).reset_index(drop=True)
    train_labels = pd.concat([train_labels, addition_train_labels], ignore_index=True).reset_index(drop=True)

    # 按照 msisdn, start_time 排序
    sort_start_time = time.time()
    train_data = train_data.sort_values(by=['msisdn', 'start_time']).reset_index(drop=True)
    train_labels = train_labels.sort_values(by=['msisdn']).reset_index(drop=True)
    print('sort time:', time.time() - sort_start_time)

    labels_aug = pd.concat([train_labels, test_labels], ignore_index=True).reindex()
    # ------------------

    # save
    print("Saving processed data...")
    os.makedirs('../self_data/processed', exist_ok=True)
    train_data.to_csv('../self_data/processed/train_data.csv', index=False)
    train_labels.to_csv('../self_data/processed/train_labels.csv', index=False)
    test_data.to_csv('../self_data/processed/test_data.csv', index=False)
    test_labels.to_csv('../self_data/processed/test_labels.csv', index=False)

    labels_aug.to_csv('../self_data/processed/labels_aug.csv', index=False)

    validation_data.to_csv('../self_data/processed/validation_data.csv', index=False)
    # TODO: test_data

else:
    print("Reading processed data...")
    train_data = pd.read_csv('../self_data/processed/train_data.csv', dtype=dtypes)
    train_labels = pd.read_csv('../self_data/processed/train_labels.csv', dtype=dtypes)
    test_data = pd.read_csv('../self_data/processed/test_data.csv', dtype=dtypes)
    test_labels = pd.read_csv('../self_data/processed/test_labels.csv', dtype=dtypes)

    labels_aug = pd.read_csv('../self_data/processed/labels_aug.csv', dtype=dtypes)

    validation_data = pd.read_csv('../self_data/processed/validation_data.csv', dtype=dtypes)

labeled_data_aug = pd.concat([train_data, test_data], ignore_index=True).reindex()
assert len(labeled_data_aug['msisdn'].unique()) == len(labels_aug['msisdn'].unique())

# 转换时间格式
labeled_data_aug['start_time'] = pd.to_datetime(labeled_data_aug['start_time'], format='%Y%m%d%H%M%S')
labeled_data_aug['end_time'] = pd.to_datetime(labeled_data_aug['end_time'], format='%Y%m%d%H%M%S')
labeled_data_aug['open_datetime'] = pd.to_datetime(labeled_data_aug['open_datetime'], format='%Y%m%d%H%M%S')
labeled_data_aug['update_time'] = pd.to_datetime(labeled_data_aug['update_time'])
labeled_data_aug['date'] = pd.to_datetime(labeled_data_aug['date'])

validation_data['start_time'] = pd.to_datetime(validation_data['start_time'], format='%Y%m%d%H%M%S')
validation_data['end_time'] = pd.to_datetime(validation_data['end_time'], format='%Y%m%d%H%M%S')
validation_data['open_datetime'] = pd.to_datetime(validation_data['open_datetime'], format='%Y%m%d%H%M%S',errors='coerce')
validation_data['update_time'] = pd.to_datetime(validation_data['update_time'])
validation_data['date'] = pd.to_datetime(validation_data['date'])

In [135]:
# 为每条记录添加start_time_diff，记录 start_time 与上一条记录的 start_time 之差 (单位：秒)
start_time_diff = labeled_data_aug.groupby('msisdn')['start_time'].diff().dt.total_seconds().fillna(0).reset_index(drop=True)
# 将该列加入到数据集中
labeled_data_aug['start_time_diff'] = start_time_diff.copy()
# time_diff_start2end = train_data.groupby('msisdn')['end_time'].diff().dt.total_seconds().fillna(0)
start_time_diff = validation_data.groupby('msisdn')['start_time'].diff().dt.total_seconds().fillna(0).reset_index(drop=True)
validation_data['start_time_diff'] = start_time_diff.copy()

数据特征处理

In [136]:
# 聚合特征
def aggregate_features(data):
    return data.groupby('msisdn').agg({
    'call_duration': [
        ('sum', 'sum'), 
        ('mean', 'mean'), 
        ('max', 'max'), 
        ('std', 'std'),
        ('quantile_25', lambda x: x.quantile(0.25)), 
        ('quantile_50', lambda x: x.quantile(0.50)), 
        ('quantile_75', lambda x: x.quantile(0.75)),
    ],
    'cfee': [
        ('sum', 'sum'),
        ('std', 'std'), 
        ('mean', 'mean'),
    ],
    'lfee': [
        ('sum', 'sum'), 
        ('mean', 'mean'),
        ('std', 'std'),
    ],
    'hour': [
        ('mean', 'mean'), 
        ('std', 'std'), 
        ('max', 'max'), 
        ('min', 'min'),
    ],
    'dayofweek': [
        ('std', 'std'), 
        ('magic', lambda x: x.value_counts().mean()), 
        ('work_day_num', lambda x: x[x.isin([1,2,3,4,5])].count()), 
        ('weekend_num', lambda x: x[x.isin([6,7])].count()),
        ('mode', lambda x: x.mode().values[0]),
        ('work_day_weekend_diff', lambda x: (x[x.isin([1,2,3,4,5])].count() - x[x.isin([6,7])].count()) / (x[x.isin([1,2,3,4,5])].count() + x[x.isin([6,7])].count())),
    ],
    # 'home_area_code': [
    #     ('home_area_code_nunique', 'nunique')
    # ],
    'visit_area_code': [
        ('nunique', 'nunique'),
        ('times_not_at_home_area', lambda x: x[x != x.shift()].count()/x.count())
    ],
    'called_home_code': [
        ('nunique', 'nunique'),
        ('called_diff_home_code', lambda x: x[x != x.shift()].count() / x.count())
    ],
    'called_code': [
        ('nunique', 'nunique'),
        ('diff', lambda x: x[x != x.shift()].count()/ x.count())
    ],
    'open_datetime': [
        ('open_count', 'nunique')
    ],
    'other_party': [
        ('account_person_num', 'nunique'),
        ('called_diff_home_code', lambda x: x[x != x.shift()].count() / x.count())
    ],
    'a_serv_type': [
        ('call_num', lambda x: x[x.isin([1, 3])].count()), 
        ('called_num', lambda x: x[x == 2].count()),
        ('call_called_normalized_diff', lambda x: (x[x.isin([1, 3])].count() - x[x == 2].count()) /  (x[x.isin([1, 3])].count() + x[x == 2].count())),
    ],
    'start_time_diff': [
        ('start_time_diff_mean', 'mean'), 
        ('start_time_diff_std', 'std'), 
        ('max', 'max'), 
        ('coefficient_of_variation', lambda x: x.std() / x.mean()),
    ], 
    # 'phone1_type': [
    #     ('nunique', 'nunique'),
    #     ('mode', lambda x: x.mode().values[0])
    # ],
    # 'distance': [
    #     ('sum', 'sum'), 
    #     ('std', 'std'), 
    #     ('max', 'max'), 
    #     ('quantile_25', lambda x: x.quantile(0.25)), 
    #     ('quantile_50', lambda x: x.quantile(0.50)), 
    #     ('quantile_75', lambda x: x.quantile(0.75)),
    # ]
})

labeled_aug_features = aggregate_features(labeled_data_aug)
validation_features = aggregate_features(validation_data)

labeled_aug_features.columns = ['+'.join(col).strip() for col in labeled_aug_features.columns.values]
validation_features.columns = ['+'.join(col).strip() for col in validation_features.columns.values]

labeled_aug_features.columns = labeled_aug_features.columns.str.replace('[', '').str.replace(']', '').str.replace('<', '').str.replace('>', '').str.replace('(', '').str.replace(')', '').str.replace(',', '').str.replace(' ', '_')
validation_features.columns = validation_features.columns.str.replace('[', '').str.replace(']', '').str.replace('<', '').str.replace('>', '').str.replace('(', '').str.replace(')', '').str.replace(',', '').str.replace(' ', '_')

# 重置索引
labeled_aug_features = labeled_aug_features.reset_index()
validation_features = validation_features.reset_index()

# 合并标签数据
labeled_aug_features = labeled_aug_features.merge(labels_aug, on='msisdn', how='left')
# 打印结果
# labeled_aug_features

# # 添加 ae 的编码特征
# labeled_ae = pd.read_csv('../data/ae/train.csv', dtype=dtypes)
# valid_ae = pd.read_csv('../data/ae/val.csv', dtype=dtypes)
# labeled_aug_features = labeled_aug_features.merge(labeled_ae, on='msisdn', how='left')
# validation_features = validation_features.merge(valid_ae, on='msisdn', how='left')

In [ ]:
labeled_aug_features.columns

In [ ]:
len(labeled_aug_features.columns)

In [ ]:
def get_nan(train):
    # 获取 train 中的 nan值
    train_nan = train[train.isnull().T.any()]
    # 统计 每列含有的 nan 数量
    for col in train.columns:
        if train[col].isnull().sum() > 0:
            print(col, train[col].isnull().sum())

    return train_nan
get_nan(labeled_aug_features), get_nan(validation_features)

In [ ]:
# 一般只有 std 会出现 nan 值故所有的 nan 值填充为 0
labeled_aug_features = labeled_aug_features.fillna(0)
validation_features = validation_features.fillna(0)

def get_nan(train):
    # 获取 train 中的 nan值
    train_nan = train[train.isnull().T.any()]
    # 统计 每列含有的 nan 数量
    for col in train.columns:
        if train[col].isnull().sum() > 0:
            print(col, train[col].isnull().sum())

    return train_nan
get_nan(labeled_aug_features), get_nan(validation_features)

In [141]:
# # # 交叉特征
# # # 将所有特征两两相乘
# from itertools import combinations
# from tqdm import tqdm

# def cross_features(data):
#     cross_features = []
#     new_features = []
#     cross_cols = data.columns.tolist()
#     rm_cols = ['msisdn', 'is_sa']
#     for col in rm_cols:
#         if col in cross_cols:
#             cross_cols.remove(col)

#     for i, j in tqdm(combinations(cross_cols, 2), total=len(cross_cols) * (len(cross_cols) - 1) // 2):
#         new_features.append(data[i] * data[j])
#         cross_features.append(f'{i}_cross_{j}')
#     new_features = pd.concat(new_features, axis=1)
#     new_features.columns = cross_features
#     data = pd.concat([data, new_features], axis=1)
#     return data, cross_features

# labeled_aug_features, _ = cross_features(labeled_aug_features)
# validation_features, _ = cross_features(validation_features)

In [ ]:
X = labeled_aug_features.drop(['msisdn'], axis=1)
y = labeled_aug_features['is_sa']
X_validation = validation_features.drop(['msisdn'], axis=1)

n_sample = y.shape[0]
n_pos_sample = y[y ==1].shape[0]
n_neg_sample = y[y == 0].shape[0]
print('样本个数：{}; 正样本占{:.2%}; 负样本占{:.2%}'.format(n_sample,
                                                   n_pos_sample / n_sample,
                                                   n_neg_sample / n_sample))
print('特征维数：', X.shape[1])

In [ ]:
labeled_aug_features.columns

In [144]:
# TODO use all_X to impute
imputer = SimpleImputer(strategy='most_frequent')
X = imputer.fit_transform(X)

In [145]:
imputer2 = SimpleImputer(strategy='most_frequent')
X_validation = imputer2.fit_transform(X_validation)

In [ ]:
X.shape

In [ ]:
X_validation.shape

In [ ]:
y.shape

In [ ]:
assert len(train_labels) + len(test_labels) == len(labeled_aug_features)
len(train_labels), len(test_labels)

In [ ]:
from sklearn.utils import shuffle
# 将 msisdn 和 is_sa 并入 X 再划分
train_data_msisdn = train_labels['msisdn']
test_data_msisdn = test_labels['msisdn']
X_df = pd.DataFrame(X, columns=labeled_aug_features.drop(['msisdn'], axis=1).columns)
X_df = pd.concat([labeled_aug_features[['msisdn']], X_df], axis=1)
train_set = X_df[X_df['msisdn'].isin(train_data_msisdn)][X_df.columns[1:]]
test_set = X_df[X_df['msisdn'].isin(test_data_msisdn)][X_df.columns[1:]]

print(f"1 samples / 0 samples in train set: {len(train_set[train_set['is_sa'] == 1])} / {len(train_set[train_set['is_sa'] == 0])}")
print(f"1 samples / 0 samples in test set: {len(test_set[test_set['is_sa'] == 1])} / {len(test_set[test_set['is_sa'] == 0])}")

if ALL:
    # if not NO_SMOTE:
    #     smote = SMOTE(random_state=42)    # 处理过采样的方法
    #     X, y = smote.fit_resample(X, y)
    # X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42, shuffle=True)
    train_len = len(test_set) + len(train_set)
    test_len = 0
else:
    # X_train,X_test,y_train,y_test = train_test_split(X,y,stratify = y,test_size= 0.3,random_state=42, shuffle=True)
    # X_train, y_train = shuffle(X_train, y_train, random_state=42)
    train_len, test_len = len(train_set), len(test_set)

    # if not NO_SMOTE:
    #     smote = SMOTE(random_state=42)    # 处理过采样的方法
    #     X_train, y_train = smote.fit_resample(X_train, y_train)
    #     print('通过SMOTE方法平衡正负样本后')
    #     n_sample = y_train.shape[0]
    #     n_pos_sample = y_train[y_train == 1].shape[0]
    #     n_neg_sample = y_train[y_train == 0].shape[0]
    #     print('样本个数：{}; 正样本占{:.2%}; 负样本占{:.2%}'.format(n_sample,
    #                                                     n_pos_sample / n_sample,
    #                                                     n_neg_sample / n_sample))
    #     print('特征维数：', X.shape[1])

In [151]:
columns = labeled_aug_features.columns.tolist()
columns.remove('msisdn')
valid_set = np.c_[X_validation, np.zeros(X_validation.shape[0])]
valid_set = pd.DataFrame(valid_set, columns=columns)
valid_set['is_sa'] = -1

In [ ]:
train_set.head()

In [ ]:
test_set.describe()

In [154]:
all_set = pd.concat([train_set, test_set, valid_set], axis=0).reset_index(drop=True)
labeled_data_len = train_set.shape[0] + test_set.shape[0]

In [ ]:
test_set.shape, train_set.shape, valid_set.shape, all_set.shape

In [156]:
NO_SMOTE = False

In [ ]:
labeled_set, valid_set = all_set.iloc[:labeled_data_len].copy(), all_set.iloc[labeled_data_len:].copy()
labeled_set.reset_index(drop=True, inplace=True)
valid_set.reset_index(drop=True, inplace=True)
# 有一些值在SMOTE后对数变换后为 NaN，需要删除这些数据
print(labeled_set.isnull().sum().sum())
labeled_set = labeled_set.dropna()
print(labeled_set.isnull().sum().sum())
assert valid_set.shape[0] == validation_features.shape[0]

# 重新划分训练集和测试集
if not ALL:
    train_set, test_set = labeled_set.iloc[:train_len].copy(), labeled_set.iloc[train_len:].copy()
    train_set.reset_index(drop=True, inplace=True)
    test_set.reset_index(drop=True, inplace=True)

    # remove_columns = ['distance_distance_std', "start_time_diff_start_time_diff_max", "distance_distance_quantile_75"]

    # remove_columns = ["lfee_lfee_std", "lfee_lfee_mean", 'call_duration_call_duration_max', "distance_distance_quantile_50", "call_duration_call_duration_quantile_25"]
    # remove_columns = ["7", "6", "lfee_lfee_mean", "hour_hour_std", "1", "call_duration_call_duration_quantile_75", "3", "cfee_cfee_std", "start_time_diff_start_time_diff_max", "call_duration_call_duration_max", "dayofweek_dayofweek_mode", "distance_distance_quantile_75", "cfee_cfee_mean"] # , "visit_area_code_visit_area_code_nunique", "visit_area_code_visit_area_code_nunique"
    # remove_columns = ['visit_area_code+nunique_cross_start_time_diff+max', "distance+std"]
    # remove_columns = ['dayofweek+std', 'start_time_diff+max', 'distance+quantile_75', 'lfee+mean', 'lfee+std', 'lfee+sum', 'cfee+sum', '6', 'visit_area_code+nunique']
    remove_columns = ['cfee+std', 'start_time_diff+start_time_diff_std', 'lfee+mean', 'lfee+sum', 'lfee+std', 'start_time_diff+max']
    train_set = train_set.drop(remove_columns, axis=1)
    test_set = test_set.drop(remove_columns, axis=1)
    valid_set = valid_set.drop(remove_columns, axis=1)
    assert train_set.shape[1] == test_set.shape[1] == valid_set.shape[1]
   
else:
    remove_columns = ['cfee+std', 'start_time_diff+start_time_diff_std', 'lfee+mean', 'lfee+sum', 'lfee+std']
    labeled_set = labeled_set.drop(remove_columns, axis=1)
    valid_set = valid_set.drop(remove_columns, axis=1)
    test_set = test_set.drop(remove_columns, axis=1)
    assert labeled_set.shape[1] == valid_set.shape[1] == test_set.shape[1]
    
# 对采样数据做 smote
if not NO_SMOTE:
    smote = SMOTE(random_state=42)    # 处理过采样的方法
    X_train, y_train = smote.fit_resample(train_set.drop(['is_sa'], axis=1), train_set['is_sa'])
    train_set = pd.concat([X_train, y_train], axis=1)
    print('通过SMOTE方法平衡正负样本后')
    n_sample = y_train.shape[0]
    n_pos_sample = y_train[y_train == 1].shape[0]
    n_neg_sample = y_train[y_train == 0].shape[0]
    print('样本个数：{}; 正样本占{:.2%}; 负样本占{:.2%}'.format(n_sample,
                                                    n_pos_sample / n_sample,
                                                    n_neg_sample / n_sample))

In [ ]:
train_set.shape, test_set.shape, valid_set.shape

In [ ]:
# 使用 autogluon 训练
from autogluon.tabular import TabularPredictor
# import ray
# 使用防止过拟合的超参数
# hyperparameters = {
#     'GBM': {'lambda_l1': 1e-2, 'lambda_l2': 1e-2},
#     'FASTAI': {'dropout_prob': 0.2}
# }
# ray.shutdown()
# ray.init(include_dashboard=True, object_store_memory=10**9)  # Increase object store memory

# 输入数据X_train, y_train
if not ALL:
    # 交叉验证训练
    model = TabularPredictor(label='is_sa', eval_metric='f1', problem_type='binary').fit(train_set, presets='medium_quality', time_limit=3600)
    # , excluded_model_types=['KNN']
    # model = TabularPredictor(label='is_sa', eval_metric='f1', problem_type='binary').fit(train_set, presets='best_quality', time_limit=3600)
else:
    model = TabularPredictor(label='is_sa', eval_metric='f1', problem_type='binary').fit(labeled_set, presets='best_quality', num_bag_folds=10, time_limit=3600)

In [ ]:
if not ALL:
    print(model.evaluate(test_set))

In [ ]:
feature_importance = model.feature_importance(test_set if not ALL else labeled_set)
print(feature_importance)
feature_importance

In [ ]:
# leaderboard
if not ALL:
    leaderboard = model.leaderboard(test_set, silent=True)
    print(leaderboard)
else:
    leaderboard = model.leaderboard(labeled_set, silent=True)
    print(leaderboard)
leaderboard

In [ ]:
# 在testset 上计算指标
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

if not ALL:
    y_pred = model.predict(test_set)
    y_true = test_set['is_sa']
    print(classification_report(y_true, y_pred))
    print(confusion_matrix(y_true, y_pred))

In [ ]:
# 模型决策阈值微调
threadhold = 0.2
if not ALL:
    y_pred_proba = model.predict_proba(test_set)
    # print(y_pred_proba)
    y_pred = (y_pred_proba.iloc[:, 1] > threadhold).astype(int)
    print(classification_report(y_true, y_pred))
    print(confusion_matrix(y_true, y_pred))

In [ ]:
valid_set

In [ ]:
test_set.shape

In [ ]:
# 预测
y_validation_pred = model.predict(valid_set.drop('is_sa', axis=1))

# 将预测结果与 msisdn 对应起来
validation_results = validation_features[['msisdn']].copy()
validation_results['is_sa'] = y_validation_pred.astype(int)

print(validation_results.describe())

# 保存结果到CSV文件
import time
time_str = time.strftime("%Y%m%d%H%M%S", time.localtime())
file_name = './valid_large_data_{}.csv'.format(time_str) if ALL else './valid_small_data_{}.csv'.format(time_str)
validation_results.to_csv(file_name, index=False)
print(file_name)

In [168]:
# import numpy as np
# from sklearn.metrics import f1_score, classification_report, confusion_matrix

# # 自动调整最佳阈值
# if not ALL:
#     y_true = test_set['is_sa']
#     y_pred_proba = model.predict_proba(test_set)

#     thresholds = np.arange(0.0, 1.0, 0.01)
#     f1_scores = []

#     for threshold in thresholds:
#         y_pred = (y_pred_proba.iloc[:, 1] >= threshold).astype(int)
#         f1 = f1_score(y_true, y_pred)
#         f1_scores.append(f1)

#     best_threshold = thresholds[np.argmax(f1_scores)]
#     print(f'最佳阈值：{best_threshold}')
#     print(f'最佳 F1 分数：{max(f1_scores)}')

#     # 使用最佳阈值进行预测
#     y_pred = (y_pred_proba.iloc[:, 1] >= best_threshold).astype(int)
#     print(classification_report(y_true, y_pred))
#     print(confusion_matrix(y_true, y_pred))

In [169]:
# # 阈值微调版结果
# best_threshold = 0.3
# # 使用最佳决策阈值进行预测
# y_validation_pred_proba = model.predict_proba(valid_set.drop('is_sa', axis=1))
# y_validation_pred = (y_validation_pred_proba.iloc[:, 1] >= best_threshold).astype(int)

# # 将预测结果与 msisdn 对应起来
# validation_results = validation_features[['msisdn']].copy()
# validation_results['is_sa'] = y_validation_pred.astype(int)

# print(validation_results.describe())

# # 保存结果到CSV文件
# import time
# time_str = time.strftime("%Y%m%d%H%M%S", time.localtime())
# file_name = './valid_large_data_{}.csv'.format(time_str) if ALL else './valid_small_data_{}.csv'.format(time_str)
# validation_results.to_csv(file_name, index=False)
# print(file_name)